# ANNDL: 1st challenge


## ⚙️ Import Libraries
Import the libraries needed for the project and fix the seed for repeatability.

In [ ]:
import numpy as np
import random

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

seed = 42
input_img_size = 96



np.random.seed(seed)
tf.random.set_seed(seed)

#############################


## ⏳ Load the Data

The data are loaded from the filtered, pre-processed data. For more details, see '''Preprocessing.ipynb''' notebook.

In [ ]:
filtered_dataset_path = './Preprocessing/balanced_dataset.npz'
data = np.load(filtered_dataset_path)
X = data['images']
y = data['labels']

# Define a mapping of labels to their corresponding digit names
labels = {0:'Basophil', 1:'Eosinophil', 2:'Erythroblast', 3:'Immature granulocytes', 4:'Lymphocyte', 5:'Monocyte', 6:'Neutrophil', 7:'Platelet'}

# Save unique labels
unique_labels = list(labels.values())

# Show the shape of the dataset
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

##  🦠 Process the Data


In [ ]:
# Display a sample of images from the training-validation dataset
num_img = 10
random_indices = random.sample(range(len(X)), num_img)

fig, axes = plt.subplots(1, num_img, figsize=(input_img_size, input_img_size))

# Iterate through the selected number of images
for i, idx in enumerate(random_indices):
    ax = axes[i % num_img]
    ax.imshow(np.squeeze(X[idx]), vmin=0., vmax=1.)
    ax.set_title(labels[y[idx]])
    ax.axis('off')

# Adjust layout and display the images
plt.tight_layout()
plt.show()

In [ ]:
# Inspect the target
print('Counting occurrences of target classes:')
print(pd.DataFrame(y, columns=['digit'])['digit'].value_counts())

In [ ]:
# Normalize the data to the range [0, 1] and encode output labels
X = (X / 255).astype('float32')
y = tfk.utils.to_categorical(y, num_classes=len(unique_labels))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)
print(f'Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}')
print(f'Training labels shape: {y_train.shape}, Validation labels shape: {y_val.shape}')

## 🔆 Agumenting the training set
We use AutoContrast from the keras library. It enhances contrast adaptively and may help in highlighting subtle differences between cell types without altering structure.

In [ ]:
# Install and import keras
! pip install keras-cv
import keras_cv as kcv

In [ ]:
# Implement AutoContrast
value_range = [0, 1]
autocontrast = kcv.layers.AutoContrast(value_range)
contrast_result = autocontrast({'images': X_train, 'labels': y_train})

# Show some images
num_img = 10
random_indices = random.sample(range(len(contrast_result["images"])), num_img)
fig, axes = plt.subplots(1, num_img, figsize=(input_img_size, input_img_size))

for i, idx in enumerate(random_indices):
    ax = axes[i % num_img]
    ax.imshow(np.squeeze(contrast_result["images"][idx]), vmin=0., vmax=1.)
    # Get the index of the maximum value (representing the predicted class)
    label_index = np.argmax(contrast_result["labels"][idx])
    ax.set_title(labels[label_index])
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Implement RandomSaturation
factor = [0, 1]
randomsaturation = kcv.layers.RandomSaturation(factor, seed=seed)
saturation_result = randomsaturation({'images': X_train, 'labels': y_train})

# Show some images
num_img = 10
random_indices = random.sample(range(len(saturation_result["images"])), num_img)
fig, axes = plt.subplots(1, num_img, figsize=(input_img_size, input_img_size))

for i, idx in enumerate(random_indices):
    ax = axes[i % num_img]
    ax.imshow(np.squeeze(saturation_result["images"][idx]), vmin=0., vmax=1.)
    # Get the index of the maximum value (representing the predicted class)
    label_index = np.argmax(saturation_result["labels"][idx])
    ax.set_title(labels[label_index])
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Add the result of the augmentatation to the original training set
X_train = np.concatenate((X_train, contrast_result["images"]), axis=0)
y_train = np.concatenate((y_train, contrast_result["labels"]), axis=0)
X_train = np.concatenate((X_train, saturation_result["images"]), axis=0)
y_train = np.concatenate((y_train, saturation_result["labels"]), axis=0)

# Inspect the target
print('Counting occurrences of target classes:')
y_train_indices = np.argmax(y_train, axis=1) #
print(pd.DataFrame(y_train_indices, columns=['digit'])['digit'].value_counts())

## 🛠️ Train and Save the Model

In [ ]:
# Hyperparameters
epochs = 10
batch_size = 10
learning_rate = 0.001

# Retrieve input and output shape
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1]

In [ ]:
def build_model (
    input_shape = input_shape,
    output_shape = output_shape,
    learning_rate = learning_rate,
    dropout_rate = 0.5,
    augmentation = None,
    flatten = True,
    seed = seed):

  tf.random.set_seed(seed)
  num_filters = 16
  default_kernel_size = 3

  # Input layer
  inputs = tfkl.Input(shape=input_shape, name='Input')

  # Apply augmentation layers, if specified, and create the first convolutional layer
  if augmentation is None:
    x = tfkl.Conv2D(filters=num_filters, kernel_size=default_kernel_size, padding='same', activation='relu', name='Conv2D_1')(inputs)
  else:
    x = augmentation(inputs)
    x = tfkl.Conv2D(filters=num_filters, kernel_size=default_kernel_size, padding='same', activation='relu', name='Conv2D_1')(x)
  x = tfkl.MaxPooling2D(pool_size=2, name='MaxPooling2D_1')(x)

  x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='Conv2D_2')(x)
  x = tfkl.ReLU(name='relu2')(x)
  x = tfkl.MaxPooling2D(name='mp2')(x)

  x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='Conv2D_3')(x)
  x = tfkl.ReLU(name='relu3')(x)
  x = tfkl.MaxPooling2D(name='mp3')(x)

  x = tfkl.Conv2D(filters=128, kernel_size=3, padding='same', name='Conv2D_4')(x)
  x = tfkl.ReLU(name='relu4')(x)
  x = tfkl.MaxPooling2D(name='mp4')(x)

  # Flatten layer
  if flatten == True:
    x = tfkl.Flatten(name='Flatten')(x)
  else:
    x = tfkl.GlobalAveragePooling2D(name='GlobalAveragePooling2D')(x)

  # Fully connected layers
  x = tfkl.Dense(units=128, activation='relu', name='Dense_1')(x)
  x = tfkl.Dropout(rate=dropout_rate, name='Dropout1')(x)

  x = tfkl.Dense(units=64, activation='relu', name='Dense_2')(x)
  x = tfkl.Dropout(rate=dropout_rate, name='Dropout2')(x)

  # Output layer
  x = tfkl.Dense(units=output_shape, name='Output')(x)
  outputs = tfkl.Activation('softmax', name='Activation_Output')(x)

  # Create the model
  model = tfk.Model(inputs=inputs, outputs=outputs, name='CNN_Model')

  # Compile the model
  loss = tfk.losses.CategoricalCrossentropy()
  optimizer = tfk.optimizers.Adam(learning_rate)
  metrics = ['accuracy']
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

  return model

In [ ]:
# Define a data augmentation pipeline
# Layer performing some geometric operations, that resemble that of the RandAugment
# layer (which was too computationally demanding to be used inside the network).
augmentation = tf.keras.Sequential ([
      tfkl.RandomFlip(mode='horizontal_and_vertical'),
      tfkl.RandomBrightness(0.2, value_range=(0,1)),
      tfkl.RandomTranslation(0.2,0.2),
      tfkl.RandomZoom(0.2),
      tfkl.RandomRotation(0.2),
      tfkl.RandomContrast(0.2)
], name='augment')

In [ ]:
# Early stopping callback
patience = 20

early_stopping = tfk.callbacks.EarlyStopping (
    monitor='val_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

callbacks = [early_stopping]

In [ ]:
# Build the model with specified input and output shapes
model = build_model(flatten=False)

# Display a summary of the model architecture
model.summary(expand_nested=True, show_trainable=True)

# Plot the model architecture
tfk.utils.plot_model(model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
history = model.fit (
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'weights.keras'
model.save(model_filename)

# Delete the model to free up resources
del model

In [ ]:
# Load the saved model
model = tfk.models.load_model(model_filename)

# Compute the confusion matrix
predictions = model.predict(X_val)
pred_classes = np.argmax(predictions, axis=-1)
true_classes = np.argmax(y_val, axis=-1)
cm = confusion_matrix(true_classes, pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, annot=True, fmt='', xticklabels=unique_labels, yticklabels=unique_labels, cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
# Plot the loss and accuracy metrics
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training set', alpha=.3, color='green', linestyle='--')
plt.plot(history['val_loss'], label='Validation set', alpha=.8, color='blue')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training set', alpha=.3, color='green', linestyle='--')
plt.plot(history['val_accuracy'], label='Validation set', alpha=.8, color='blue')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('weights.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        X = X / 255.0
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds